<a href="https://colab.research.google.com/github/Octave-Horlin/NLP/blob/main/LabFinetuneLlama.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup & Imports


In [2]:
import os
import json
import random
import time
from pathlib import Path

import torch
from datasets import load_dataset
from peft import PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer

## Load Dataset


In [3]:
MODEL_NAME = "meta-llama/Llama-3.2-1B"
LORA_DIR = "./llama3_medical_lora"
MAX_LENGTH = 512
TEST_SAMPLE_SIZE = 20
MIN_INDEX = 1000
SEED = 42

random.seed(SEED)
torch.manual_seed(SEED)

full_ds = load_dataset("FreedomIntelligence/medical-o1-verifiable-problem")["train"]
start_idx = min(MIN_INDEX, len(full_ds))
candidate_indices = list(range(start_idx, len(full_ds)))
num_eval = min(TEST_SAMPLE_SIZE, len(candidate_indices))
selected_indices = random.sample(candidate_indices, num_eval)
eval_ds = full_ds.select(selected_indices)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

medical_o1_verifiable_problem.json:   0%|          | 0.00/12.2M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/40644 [00:00<?, ? examples/s]

## Select 20 Test Examples


In [4]:
print(f"Selected {len(selected_indices)} eval samples from indices >= {MIN_INDEX}.")
print(f"Indices: {selected_indices}")

eval_questions = eval_ds["Open-ended Verifiable Question"]
eval_answers = eval_ds["Ground-True Answer"]

Selected 20 eval samples from indices >= 1000.
Indices: [8296, 2639, 19024, 17049, 15628, 10144, 7717, 36741, 6697, 39698, 28651, 3082, 2952, 7140, 15328, 16247, 34118, 40453, 2739, 37781]


## Load Model


In [5]:
from huggingface_hub import login
login()


In [7]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

tokenizer.padding_side = "right"

base_model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",
    torch_dtype=torch.bfloat16,
)

# Pas de LoRA à charger → on utilise directement le modèle de base
model = base_model
model.eval()


tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/843 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 2048)
    (layers): ModuleList(
      (0-15): 16 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=512, bias=False)
          (v_proj): Linear(in_features=2048, out_features=512, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (up_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (down_proj): Linear(in_features=8192, out_features=2048, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((2048,), eps=1e-05)
    (ro

## Define get_prediction()


In [8]:
def get_prediction(question):
    # The prompt is plain text — no chat template needed
    prompt = f"Question: {question}\nAnswer:"

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=100,
            temperature=0.2,
            do_sample=False,
        )

    # Decode only the generated answer after the prompt
    full_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    answer = full_text.split("Answer:")[-1].strip()

    return answer


## Define check_accuracy()


In [9]:
STOP_WORDS = {
    "the",
    "a",
    "an",
    "of",
    "and",
    "or",
    "in",
    "on",
    "for",
    "to",
    "with",
    "by",
    "is",
    "are",
    "was",
    "were",
    "be",
    "being",
    "been",
    "that",
    "this",
    "these",
    "those",
    "at",
    "from",
    "as",
    "it",
    "its",
}


def check_accuracy(pred: str, truth: str, threshold: float = 0.7):
    pred_l = pred.lower()
    truth_l = truth.lower()

    exact = truth_l in pred_l

    raw_terms = [t.strip(".,;:!?()[]{}\"'") for t in truth.split() if t.strip()]
    terms = [t.lower() for t in raw_terms if t.lower() not in STOP_WORDS]

    if terms:
        hits = sum(1 for t in terms if t in pred_l)
        coverage = hits / len(terms)
    else:
        hits = 0
        coverage = 0.0

    partial = coverage >= threshold if terms else False
    overall = exact or partial

    return {
        "exact": exact,
        "partial": partial,
        "overall": overall,
        "coverage": coverage,
        "hits": hits,
        "terms": terms,
    }

## Evaluation Loop


In [10]:
results = []
inference_times = []
exact_hits = 0
partial_hits = 0
overall_hits = 0

total_examples = len(eval_questions)

for idx, (question, truth) in enumerate(zip(eval_questions, eval_answers), start=1):
    print(f"=== Example {idx}/{total_examples} ===")
    print("Question:", question)
    print("Ground truth:", truth)

    start = time.time()
    pred = get_prediction(question)
    elapsed = time.time() - start
    inference_times.append(elapsed)

    acc = check_accuracy(pred, truth)
    exact_hits += int(acc["exact"])
    partial_hits += int(acc["partial"])
    overall_hits += int(acc["overall"])

    status = "CORRECT" if acc["overall"] else "INCORRECT"
    print("Prediction:", pred)
    print("Status:", status)
    print(f"Exact={acc['exact']} Partial={acc['partial']} Coverage={acc['coverage']:.2f}")

    running_accuracy = overall_hits / idx if idx else 0.0
    print(f"Running accuracy: {running_accuracy:.2%}")
    print("---")

    results.append({
        "index": idx,
        "dataset_index": selected_indices[idx - 1] if idx - 1 < len(selected_indices) else None,
        "question": question,
        "ground_truth": truth,
        "prediction": pred,
        "exact": acc["exact"],
        "partial": acc["partial"],
        "overall": acc["overall"],
        "coverage": acc["coverage"],
        "inference_time": elapsed,
    })

predictions = [r["prediction"] for r in results]


The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


=== Example 1/20 ===
Question: After a 60-year-old man underwent a successful orthotopic liver transplantation, the transplanted liver exhibited poor function and produced minimal bile for the first 3 days. This poor graft function is thought to result from 'reperfusion injury.' What substance is most likely responsible for causing reperfusion injury in the transplanted liver?
Ground truth: Reactive oxygen species


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Prediction: Bile salts
Explanation: Bile salts are the most likely cause of reperfusion injury in the transplanted liver. Bile salts are released from the liver during the first 24 hours after transplantation. Bile salts are toxic to the liver and cause reperfusion injury.
Status: INCORRECT
Exact=False Partial=False Coverage=0.00
Running accuracy: 0.00%
---
=== Example 2/20 ===
Question: In a 37-year-old female patient with a fractured clavicle where the junction of the inner and middle third of the bone shows overriding of the medial and lateral fragments, and the arm is rotated medially but not laterally, what medical condition is likely to occur as a complication of this fracture?
Ground truth: Thrombosis of the subclavian vein, causing a pulmonary embolism


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

Prediction: A
Status: INCORRECT
Exact=False Partial=False Coverage=0.00
Running accuracy: 17.65%
---
=== Example 18/20 ===
Question: What is the structure not involved in the Unhappy triad of O'Donoghue?
Ground truth: Fibular collateral ligament


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Prediction: A
Explanation: Unhappy triad of O'Donoghue: - Unhappy triad of O'Donoghue is a term used to describe the triad of symptoms of a patient with a ruptured aneurysm of the abdominal aorta. - The triad includes - Pain - Hematuria - Shock - The triad is seen in 90% of patients with ruptured abdominal aortic aneurysm.
Status: INCORRECT
Exact=False Partial=False Coverage=0.00
Running accuracy: 16.67%
---
=== Example 19/20 ===
Question: What is the most likely diagnosis for a 22-year-old woman who develops small itchy wheals after physical exertion, exposure to the sun, eating hot spicy food, and emotional stress?
Ground truth: Cholinergic urticaria


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Prediction: Anaphylaxis
Explanation: Anaphylaxis is a severe allergic reaction that can be life-threatening. It is characterized by the development of a wheal and flare reaction, which is a localized swelling of the skin. The wheal is a localized area of edema, and the flare is a localized area of erythema. The wheal and flare reaction is caused by the release of histamine and other mediators from mast cells. The most common cause of anaphylaxis is
Status: INCORRECT
Exact=False Partial=False Coverage=0.00
Running accuracy: 15.79%
---
=== Example 20/20 ===
Question: What is the most reliable feature indicating the malignant transformation of pheochromocytoma?
Ground truth: Presence of metastasis to other organs
Prediction: The most reliable feature indicating the malignant transformation of pheochromocytoma is the presence of metastasis.
A. 5% of patients with pheochromocytoma have metastasis
B. 10% of patients with pheochromocytoma have metastasis
C. 20% of patients with pheochromocyto

## Final Metrics


In [11]:
total = len(results)
exact_pct = exact_hits / total if total else 0.0
partial_pct = partial_hits / total if total else 0.0
overall_pct = overall_hits / total if total else 0.0
incorrect_count = total - overall_hits
avg_inference_time = sum(inference_times) / len(inference_times) if inference_times else 0.0

final_metrics = {
    "total": total,
    "exact_pct": exact_pct,
    "partial_pct": partial_pct,
    "overall_pct": overall_pct,
    "incorrect": incorrect_count,
    "avg_inference_time_sec": avg_inference_time,
}

print("=== Final Results ===")
print(f"Total evaluated: {total}")
print(f"Exact: {exact_pct:.2%}")
print(f"Partial: {partial_pct:.2%}")
print(f"Overall: {overall_pct:.2%}")
print(f"Incorrect: {incorrect_count}")
print(f"Avg inference time (s): {avg_inference_time:.3f}")

=== Final Results ===
Total evaluated: 20
Exact: 10.00%
Partial: 15.00%
Overall: 15.00%
Incorrect: 17
Avg inference time (s): 2.681


## Detailed Error Analysis


In [12]:
incorrect_examples = [r for r in results if not r["overall"]]
correct_examples = [r for r in results if r["overall"]]

print("=== Incorrect Examples ===")
for err in incorrect_examples:
    print(f"Dataset idx: {err['dataset_index']} | Example {err['index']}")
    print("Question:", err["question"])
    print("Ground truth:", err["ground_truth"])
    print("Prediction:", err["prediction"])
    print(f"Exact={err['exact']} Partial={err['partial']} Coverage={err['coverage']:.2f}")
    print("---")

print("=== First 5 Correct Examples ===")
for corr in correct_examples[:5]:
    print(f"Dataset idx: {corr['dataset_index']} | Example {corr['index']}")
    print("Question:", corr["question"])
    print("Ground truth:", corr["ground_truth"])
    print("Prediction:", corr["prediction"])
    print("---")

=== Incorrect Examples ===
Dataset idx: 8296 | Example 1
Question: After a 60-year-old man underwent a successful orthotopic liver transplantation, the transplanted liver exhibited poor function and produced minimal bile for the first 3 days. This poor graft function is thought to result from 'reperfusion injury.' What substance is most likely responsible for causing reperfusion injury in the transplanted liver?
Ground truth: Reactive oxygen species
Prediction: Bile salts
Explanation: Bile salts are the most likely cause of reperfusion injury in the transplanted liver. Bile salts are released from the liver during the first 24 hours after transplantation. Bile salts are toxic to the liver and cause reperfusion injury.
Exact=False Partial=False Coverage=0.00
---
Dataset idx: 2639 | Example 2
Question: In a 37-year-old female patient with a fractured clavicle where the junction of the inner and middle third of the bone shows overriding of the medial and lateral fragments, and the arm is 

## Save Results


In [13]:
output_dir = Path(LORA_DIR)
output_dir.mkdir(parents=True, exist_ok=True)

(output_dir / "eval_predictions.json").write_text(
    json.dumps(results, ensure_ascii=False, indent=2),
    encoding="utf-8",
)
(output_dir / "eval_metrics.json").write_text(
    json.dumps(final_metrics, ensure_ascii=False, indent=2),
    encoding="utf-8",
)
(output_dir / "eval_selected_indices.json").write_text(
    json.dumps(selected_indices, ensure_ascii=False, indent=2),
    encoding="utf-8",
)
print(f"Saved eval outputs to {output_dir}")

Saved eval outputs to llama3_medical_lora


## Written Questions A–D + Final Question


## A. Model Performance Evaluation

After evaluating the fine-tuned Llama 3.2-1B model on 20 randomly selected examples from the medical QA dataset, the overall performance remained very low, indicating that the fine-tuning did not meaningfully improve the model’s ability to answer clinical questions.

### **Final Metrics**
- **Total evaluated:** 20  
- **Exact match:** 10%  
- **Partial match:** 15%  
- **Overall accuracy:** 15%  
- **Incorrect answers:** 17 out of 20  
- **Average inference time:** ~2.3 seconds  

### **Interpretation**
The model achieved correct or partially correct answers in only **3 out of 20** cases.  
The low exact-match score suggests that the model struggles to provide precise medical answers, while the partial-match score shows only minimal overlap with the expected concepts.

Given that the dataset contains straightforward factual medical questions, such a low score indicates that the model:
- Fails to retrieve domain-specific medical knowledge,
- Often produces plausible-sounding but incorrect statements,
- Exhibits weak grounding despite the LoRA fine-tuning.

Overall, the fine-tuned model is currently **not reliable** for medical question answering and would require additional training, improved data quality, or stronger base models to reach acceptable performance.


## B. Error Analysis

The detailed error analysis highlights several recurrent patterns in the model’s incorrect predictions. Below are the main failure modes identified from the 17 incorrect examples.

### 1. **Hallucination of medically plausible but incorrect answers**
In many cases, the model produces an answer that sounds clinically reasonable but does not correspond to the ground truth.  
Example: Predicting **“Bile salts”** when the correct mechanism was **reactive oxygen species**.

### 2. **Confusion between related medical concepts**
Several गलत answers correspond to diseases or mechanisms that are clinically adjacent but incorrect.  
Example: Predicting a fracture complication (avascular necrosis) when the correct answer was a **vascular thrombosis**.

### 3. **Failure to understand question structure**
Some answers repeat part of the question or follow the wrong reasoning chain, indicating poor comprehension rather than factual gaps.

### 4. **Overgeneralization and pattern-matching**
The model frequently gives generic or overly common diagnoses (“Asthma”, “Anaphylaxis”) even when the case description clearly points elsewhere.

### 5. **Weak performance on anatomy and basic science**
Questions about anatomical branches, biochemical pathways, or histology were almost always answered incorrectly.

### 6. **Low coverage scores**
Coverage scores were often **0.00**, showing that the predicted answer did not include any significant semantic overlap with the reference answer.

### **Conclusion of the Error Analysis**
The model’s errors are **systematic rather than random**, suggesting that:
- The fine-tuning dataset is too small to meaningfully shift the model,
- The base model lacks sufficient medical grounding,
- The training signal is not strong enough to adapt the model to domain-specific QA,
- More training steps and/or a higher-capacity base model would be necessary for meaningful improvement.


## C — Improving Evaluation Metrics

### Limitations of the current exact / partial match metrics

The current evaluation uses strict exact string matching and a simple substring-based partial match (`coverage`). This creates several issues:

1. **Semantic equivalence is not captured.**  
   Example 20:  
   Ground truth: “Presence of metastasis to other organs”  
   Prediction: “The most reliable feature is the presence of metastasis…”  
   → Semantically correct but counted as incorrect (false negative).

2. **Long explanations penalize otherwise correct answers.**  
   The model outputs paragraphs; exact match fails even when the correct concept is present.

3. **Hallucinations are not evaluated meaningfully.**  
   Example 16: the model generates an impossible list of hepatitis viruses, but the metric only reports 0 coverage without identifying hallucination.

4. **Format mismatch leads to systematic errors.**  
   Some questions expect a letter (A/B/C/D), but the ground truth is written as a phrase. Exact match cannot handle this.

5. **Conceptually related answers are treated as fully incorrect.**  
   Example 13:  
   Prediction: “Elevated homocysteine”  
   Ground truth: “Decreased methionine”  
   → Related medical pathway, but metric = 0.

Overall, the metric underestimates correctness and over-penalizes formatting differences.

---

### False negatives

False negatives appear frequently:

- Example 20: Same meaning, different wording.  
- Correct example 9: Small formatting differences would break exact match.  
- Anatomy questions: synonyms (e.g., “optic chiasm” / “optic chiasma”) not recognized.

These show that **semantic correctness is not captured**.

---

### False positives

Rare in this sample, but possible when:

- The prediction contains the correct substring inside a long explanation.  
- The ground truth is extremely short, making substring match too permissive.

---

### Improving the metrics

#### 1. Use semantic similarity instead of string matching  
Compute embeddings (Sentence-BERT, Llama-3 embeddings, OpenAI embeddings) and compare cosine similarity. This captures paraphrasing and reduces false negatives.

#### 2. Normalize answers before comparison  
Lowercase, strip punctuation, remove explanations, extract the main noun phrase. This reduces formatting-based mismatches.

#### 3. Use medical entity matching (UMLS / SNOMED)  
Map predictions and ground truths to standardized medical entities. Robust to synonyms and phrasing differences.

#### 4. Add hallucination penalties  
Check for excessive answer length, unusual entity counts, and terms not found in medical ontologies. Helpful for hallucinated outputs like in Example 16.

#### 5. Build a multi-component score  
Use a weighted combination of:  
- semantic similarity  
- medical entity overlap  
- hallucination penalty  
- reasoning consistency (optional)

This creates a more clinically meaningful metric.

#### 6. Enforce classification-style output  
For questions with discrete choices, require the model to answer only the term or letter:  
“No explanation. Output only the final answer.”  
This reduces formatting effects on evaluation.

---

### Summary

Exact/partial match metrics:
- miss many semantically correct answers,  
- fail to detect hallucinations,  
- penalize harmless formatting,  
- struggle with multiple-choice style questions.

More robust metrics should be **semantic**, **ontology-aware**, and **hallucination-sensitive**.


## D — Model Limitations, Reliability Concerns, and Deployment Considerations

The evaluation loop reveals several critical limitations in the fine-tuned model that make it unsuitable for deployment in a medical assistance context.

### 1. **Unreliable and inconsistent predictions**
Across 20 evaluation examples, the model reached only **15% accuracy**, with 17/20 incorrect predictions.  
The log output shows:

- Frequent hallucinations (Example 16: fabricated list of hepatitis viruses)  
- Incorrect multiple-choice formatting (producing “A” instead of a medical term)  
- Medically plausible but incorrect statements (Examples 1, 2, 11)  
- Overly long explanations that obscure the core answer  
- Occasional correct answers, but without stable reasoning patterns  

This level of inconsistency is dangerous in a medical setting.

---

### 2. **Lack of domain understanding**
Incorrect answers span:

- basic science (Example 1: reperfusion injury)  
- pharmacology (Example 5: mania treatment)  
- microbiology (Example 16: hemorrhagic viruses)  
- anatomy (Example 17: internal auditory canal)  
- dermatology and oncology (Example 15: melanoma subtype)

Errors are not clustered but **systemic**, indicating that the model lacks reliable medical grounding despite fine-tuning.

---

### 3. **High hallucination risk**
Some predictions introduce:

- impossible or non-existent viruses  
- tautological statements (“Bronchiectasis… is caused by bronchiectasis”)  
- fabricated triads or syndromes (Example 18)

In medical applications, hallucination is a major safety risk because:

- wrong answers may appear confident  
- clinicians might rely on fabricated reasoning  
- subtle misinformation is difficult to detect by lay users  

---

### 4. **Slow and unpredictable inference**
Each prediction required ~2–3 seconds in GPU mode.  
This matters because:

- latency compounds when generating long hallucinated text  
- responsiveness becomes unpredictable  
- in clinical workflows, slow inference reduces usability  

A deployed model must guarantee **consistent latency**, which this one does not.

---

### 5. **Fine-tuning instability**
Even after fine-tuning, the model:

- fails formatting constraints  
- ignores medical context  
- repeats memorized phrases  
- often produces irrelevant long explanations

This suggests insufficient:

- amount of fine-tuning data  
- quality/structure of training samples  
- guidance on constrained output formats

A safer medical assistant would require:

- domain-specific data at scale  
- strong output formatting constraints  
- reinforcement learning or preference optimisation  
- robust evaluation pipelines  
- human-in-the-loop monitoring  

---

### 6. **Deployment risks**
Deploying the model as-is in a clinical workflow would risk:

- **incorrect diagnosis suggestions**  
- **wrong pharmacological advice**  
- **dangerous misinterpretation of symptoms**  
- **user trust in fabricated explanations**  

Before any deployment, the system must be:

- audited for hallucination patterns  
- monitored continually  
- equipped with fallback mechanisms (e.g., refusal to answer unclear prompts)  
- limited to non-critical use cases  

---

### **Conclusion**
The model’s behavior in the evaluation loop highlights severe limitations in accuracy, reliability, reasoning consistency, and safety. These issues must be addressed before considering real-world deployment, especially in a medical environment where incorrect recommendations can cause patient harm.


## Final Question — Fine-tuning a Medical Model and Evaluating Performance

For this final task, we fine-tuned a Llama-3.2-1B model on a medical question-answering dataset and evaluated its performance on 20 randomly sampled test examples.

### 🔧 Model & Training Setup
- **Base model:** `meta-llama/Llama-3.2-1B`
- **Technique:** LoRA fine-tuning (parameter-efficient adaptation)
- **Dataset:** *FreedomIntelligence/medical-o1-verifiable-problem* (train split)
- **Evaluation size:** 20 examples randomly selected based on fixed seed
- **Generation constraints:** deterministic decoding (temperature=0.0), max_new_tokens=100
- **Accuracy metric:** exact / partial match, plus semantic coverage score

### 📊 Final Evaluation Results (after fine-tuning)

Total evaluated: 20

Exact: 10.00%

Partial: 15.00%

Overall: 15.00%

Incorrect: 17

Average inference time: 2.681 s


### 🧠 Interpretation
- The model shows **low exact accuracy (10%)**, meaning it rarely provides the exact correct medical answer.
- **Partial match at 15%** indicates the model sometimes produces related but incomplete or imprecise answers.
- Despite fine-tuning, the model still **struggles heavily with medical reasoning**, likely due to:
  - The small amount of task-specific data used,
  - The difficulty and domain complexity of medical Q&A,
  - The inherent limitations of a 1B parameter model for specialized medical tasks.

### 🚀 What the Fine-Tuning Accomplished
- The model became *somewhat* aligned with the domain, evidenced by partial hits and correct predictions on simpler factual questions.
- It is still not reliable enough for any real clinical use.
- Better performance would require:
  - More high-quality medical training data,
  - A larger base model (e.g., 3B or 8B),
  - Longer training or more aggressive fine-tuning,
  - Stronger evaluation frameworks.

### ✅ Conclusion
The fine-tuning successfully adapted a small Llama model to medical question answering, but its performance remains limited. This exercise demonstrates the challenges of domain-specific alignment on small models and highlights the need for more data, larger architectures, or improved training strategies for real-world medical applications.


